# Tensorflow Tutorial revisited
We used [tensorflow tutorial of try-tf](https://github.com/jasonbaldridge/try-tf) as 
an example in old tutorial.   After tensorflow 1.4, the estimator API has made 
tensorflow significately eaiseer to use.   Here we revisit the example, to use tf 1.4.

The following just use the Saturn data as example.   Linear/Moon should be trivial to add.

In [36]:
# As usural, we open a database connection.   
import dg.conn
import dg.xtable
from dg.dsutil.csv import CsvXt

con = dg.conn.Conn(user="ftian")
con.conn.autocommit = True
print (con.ver)

4


In [3]:
# Load training data into a table
# Just run this once
saturn_train = CsvXt('https://raw.githubusercontent.com/jasonbaldridge/try-tf/master/simdata/saturn_data_train.csv')
saturn_train.add_col('tag', 'int')
saturn_train.add_col('x', 'float4')
saturn_train.add_col('y', 'float4')
xt = saturn_train.xtable(con)
xt.ctas('saturn_train')

In [4]:
# Load eval data
# Just run this once
saturn_eval = CsvXt('https://raw.githubusercontent.com/jasonbaldridge/try-tf/master/simdata/saturn_data_eval.csv')
saturn_eval.add_col('tag', 'int')
saturn_eval.add_col('x', 'float4')
saturn_eval.add_col('y', 'float4')
xt = saturn_eval.xtable(con)
xt.ctas('saturn_eval')

In [24]:
# Load data from table
xt1 = dg.xtable.fromTable(con, 'saturn_train')
xt2 = dg.xtable.fromTable(con, 'saturn_eval')

In [25]:
# Build estimator:
import dg.tf.estimator
e = dg.tf.estimator.Estimator()
e.add_out_col('data_tag', 'int')
e.add_out_col('data_x', 'float4')
e.add_out_col('data_y', 'float4')
e.add_out_col('prediction', 'int')

# Adding input
e.tfinput.add_xt(xt1, repeat=10)
e.tfinput.add_xt(xt2)

In [34]:
# Tensorflow code.   Build columns, estimator are both trivial.
# Just hook up the estimator with table input.

tfcode = """
def build_model_columns():
    x = tf.feature_column.numeric_column('x')
    y = tf.feature_column.numeric_column('y')
    return [x, y]
    
def build_estimator():
    cols = build_model_columns()
    hidden_units = [10, 10]
    run_config = tf.estimator.RunConfig().replace(
        session_config=tf.ConfigProto(device_count={'GPU': 0})
    )
    
    return tf.estimator.DNNClassifier(
        model_dir='/tmp/saturn_dnn',
        feature_columns = cols,
        hidden_units = hidden_units,
        config = run_config
    )
    
def input_fn(ii, cache_rs=False):
    features = sql_input_fn(ii, cache_rs)
    label = features.pop('tag')
    return features, tf.equal(label, 1)

def main(unused_args):
    model = build_estimator()
    model.train(input_fn=lambda: input_fn(0))
    sql_clear_cached_rs()
    results = model.predict(input_fn=lambda: input_fn(1, cache_rs=True))
    data = sql_cached_rs()
    idx = 0
    for res in results:
        resval = res['class_ids'][0]
        rec = data[idx][:]
        rec.append(resval)
        vitessedata.phi.WriteOutput(rec)
        idx += 1
    vitessedata.phi.WriteOutput(None)
"""

In [37]:
# If previous transaction failed, the con transaction state is active, but failed.
# Any further query will fail.   Just rollback ...
con.conn.commit()
e.add_tf_code(tfcode)
ext = e.build_xt(con)
print(ext.show())

+------------+--------------+-------------+--------------+
|   data_tag |       data_x |      data_y |   prediction |
|------------+--------------+-------------+--------------|
|          0 |  -2.95364    |   0.424072  |            0 |
|          1 |   9.05324    |   3.83323   |            1 |
|          1 |  -9.41899    |  -5.15292   |            1 |
|          0 |   0.0259142  |   0.013177  |            0 |
|          1 |  -5.98582    |   9.74873   |            1 |
|          1 |  -7.49102    |  -5.48394   |            1 |
|          1 |   8.6591     |  -2.32603   |            1 |
|          1 |   8.26922    |  -6.56763   |            1 |
|          0 |   2.40183    |   1.10657   |            0 |
|          0 |  -0.465756   |  -0.807678  |            0 |
|          1 |  10.2847     |   0.828401  |            1 |
|          0 |   1.88927    |  -3.06093   |            0 |
|          1 |   6.04468    |   7.74141   |            1 |
|          1 |   9.61871    |  -3.80296   |            1